In [29]:
import numpy as np
import CoolProp as CP
import matplotlib.pyplot as plt
from CoolProp.HumidAirProp import HAPropsSI
from CoolProp.CoolProp import PropsSI
import pandas as pd
from generateThermocline import generateThermocline
import torch

In [30]:
# 1 degree = 111139 m
def degreeToMeters(deg):
    return deg*111139

In [31]:
# find dead state of thermocline
def findDeadStateIndex(tempDf, tempCutoff):
    TLast = tempDf.min()["Temperature"]
    for i in range(len(tempDf)):
        if tempDf.loc[i, "Temperature"] < (TLast + tempCutoff):
            break

    return i

In [32]:
# calculates exergy for a chunk of water in the ocean from depth 1 to depth 2
def calculateSingleExergy(depthIncr, TTop, TDead, areaIncrMeter):
    P0 = 101325 # Pa
    rho = 1000     # kg/m^3
    hTop = PropsSI('H', 'T', TTop+273.153, 'P', P0, 'Water')      #enthalpy of surface water [J/kg]
    hDead = PropsSI('H', 'T', TDead+273.153, 'P', P0, 'Water')       #minimum possible enthalpy of the surface water [J/kg]
    sTop = PropsSI('S', 'T', TTop+273.153, 'P', P0, 'Water')      #entropy of surface water [J/kg-K]
    sDead = PropsSI('S', 'T', TDead+273.153, 'P', P0, 'Water')       #minimum possible entropy of the surface water [J/kg-K]

    specificExergy = (hTop - hDead) - (TDead+273.153)*(sTop - sDead) 
    exergy = specificExergy * depthIncr * rho * areaIncrMeter**2

    return exergy

In [33]:
# calculates total exergy for current location and returns total exergy + thermocline depth/dead state depth based on criteria
def calculateTotalExergy(tempDf, depthIncr, areaIncr, tempCutoff):
    totalExergy = 0
    areaIncrMeter = degreeToMeters(areaIncr)
    deadStateIndex = findDeadStateIndex(tempDf, tempCutoff)
    for i in range(deadStateIndex): # not including dead state because dead state to itself = 0 exergy
        TTop = tempDf.loc[i, "Temperature"]
        TDead = tempDf.loc[deadStateIndex, "Temperature"]
    
        exergy = calculateSingleExergy(depthIncr, TTop, TDead, areaIncrMeter)
        totalExergy += exergy

    return totalExergy, deadStateIndex

In [4]:
from pipeline import Net
modelPath = "OTEC_miniBatchState.pth"
net = Net()
net.load_state_dict(torch.load(modelPath))
net.eval()
tempDf = generateThermocline(0,0,2455562.5,2000,1,net,"Results/ThermoclinePlots")
print(tempDf)
# totalExergy = calculateTotalExergy(tempDf, 1, 5, 2)


      Depth  Temperature
0         0    27.245529
1         1    27.142408
2         2    27.039690
3         3    26.937351
4         4    26.835415
...     ...          ...
1995   1995     3.510484
1996   1996     3.509444
1997   1997     3.508404
1998   1998     3.507365
1999   1999     3.506326

[2000 rows x 2 columns]


In [34]:
findDeadStateIndex(tempDf, 3)
tempDf.loc[613,"Temperature"]

6.500275135040283

In [35]:
calculateTotalExergy(tempDf, 1, 5, 3)

(8.192142186781014e+19, 613)

In [26]:
# print(calculateSingleExergy(0, 1, 1, 27.245529, 27.142408, 5))
P0 = 101325 # Pa
rho = 1000     # kg/m^3
hTop = PropsSI('H', 'T', 27.2455+273.153, 'P', P0, 'Water')      #enthalpy of surface water [J/kg]
hDead = PropsSI('H', 'T', 6.5+273.153, 'P', P0, 'Water')       #minimum possible enthalpy of the surface water [J/kg]
sTop = PropsSI('S', 'T', 27.2455+273.153, 'P', P0, 'Water')      #entropy of surface water [J/kg-K]
sDead = PropsSI('S', 'T', 6.5+273.153, 'P', P0, 'Water')       #minimum possible entropy of the surface water [J/kg-K]

specificExergy = (hTop - hDead) - (6.5+273.153)*(sTop - sDead) 
# print((hTop - hBot))
# print((27.142408+273.153)*(sTop - sBot))
print(specificExergy * 1 * 1000 * 5**2)
# negative exergy why? **

76729670.36462615


In [10]:
(114320.97892446494-113898.90306505683)-(27.142408+273.153)*(398.61198050595544-397.1739081282055)

-9.770672001842115

In [9]:
(114.319-113.897-(27.245529+273.153)*(0.3986-0.3972)) *1000

1.4420593999933673